# Job Application AI Agent - Configuration Setup
## config/settings.py & config/logging_config.py

This notebook sets up all configuration for the Job Application AI Agent.

**Purpose**: Centralized configuration management, environment variable loading, and logging setup.


## Installation & Setup

In [ ]:
# Install required packages
!pip install python-dotenv loguru PyYAML -q
print("✓ Setup packages installed")

## 1. Settings Configuration Module

In [ ]:
import os
from typing import Dict, List
import json
from datetime import datetime

# Try importing python-dotenv, use fallback if not available
try:
    from dotenv import load_dotenv
except ImportError:
    def load_dotenv():
        pass

# Load environment variables
load_dotenv()

class Settings:
    """
    Centralized configuration management.
    Loads settings from environment variables with sensible defaults.
    """
    
    # Scraping settings
    MAX_JOBS = int(os.environ.get('MAX_JOBS', 50))
    JOB_BOARDS = ['linkedin', 'indeed', 'glassdoor']
    SCRAPING_TIMEOUT = 30
    
    # ML Classification settings
    RELEVANCE_THRESHOLD = float(os.environ.get('RELEVANCE_THRESHOLD', 0.7))
    CLASSIFIER_MODEL_PATH = 'models/pretrained_classifier.pkl'
    CLASSIFIER_INPUT_DIM = 300
    CLASSIFIER_HIDDEN_DIM = 128
    
    # LLM settings
    LLM_PROVIDER = os.environ.get('LLM_PROVIDER', 'mayini')  # mayini, openai, groq
    LLM_MODEL = os.environ.get('LLM_MODEL', 'mayini-custom')
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', '')
    GROQ_API_KEY = os.environ.get('GROQ_API_KEY', '')
    
    # MAYINI LLM settings
    MAYINI_VOCAB_SIZE = 5000
    MAYINI_HIDDEN_DIM = 256
    MAYINI_NUM_LAYERS = 2
    MAYINI_LEARNING_RATE = 0.001
    MAYINI_EPOCHS = 50
    MAYINI_BATCH_SIZE = 32
    
    # Hugging Face settings
    HF_TOKEN = os.environ.get('HF_TOKEN', '')
    HF_REPO_ID = os.environ.get('HF_REPO_ID', '')
    
    # Vector database settings
    VECTOR_DB_TYPE = os.environ.get('VECTOR_DB', 'chroma')  # chroma, pinecone
    CHROMA_PERSIST_DIR = './chroma_db'
    
    # Gradio settings
    DEBUG = os.environ.get('DEBUG', 'False').lower() == 'true'
    GRADIO_SHARE = False
    GRADIO_SERVER_NAME = '0.0.0.0'
    GRADIO_SERVER_PORT = 7860
    
    # Logging settings
    LOG_LEVEL = os.environ.get('LOG_LEVEL', 'INFO')
    LOG_FILE = os.environ.get('LOG_FILE', './logs/app.log')
    
    @classmethod
    def to_dict(cls) -> Dict:
        """Convert settings to dictionary"""
        return {key: getattr(cls, key) for key in dir(cls) 
                if not key.startswith('_') and key.isupper()}
    
    @classmethod
    def validate(cls) -> List[str]:
        """Validate critical settings"""
        errors = []
        
        if cls.LLM_PROVIDER == 'openai' and not cls.OPENAI_API_KEY:
            errors.append("OPENAI_API_KEY not set in environment")
        
        if cls.LLM_PROVIDER == 'groq' and not cls.GROQ_API_KEY:
            errors.append("GROQ_API_KEY not set in environment")
        
        if cls.RELEVANCE_THRESHOLD < 0 or cls.RELEVANCE_THRESHOLD > 1:
            errors.append("RELEVANCE_THRESHOLD must be between 0 and 1")
        
        return errors
    
    @classmethod
    def print_summary(cls):
        """Print settings summary"""
        print("\n" + "="*60)
        print("CONFIGURATION SUMMARY")
        print("="*60)
        print(f"LLM Provider: {cls.LLM_PROVIDER}")
        print(f"Relevance Threshold: {cls.RELEVANCE_THRESHOLD:.0%}")
        print(f"Max Jobs: {cls.MAX_JOBS}")
        print(f"Debug Mode: {cls.DEBUG}")
        print(f"Log Level: {cls.LOG_LEVEL}")
        print("="*60 + "\n")

# Create instance for easier access
settings = Settings()

print("✓ Settings class loaded")
settings.print_summary()

## 2. Logging Configuration Module

In [ ]:
from loguru import logger
import sys
from pathlib import Path

class LoggingConfig:
    """
    Configure logging for the entire application.
    Uses loguru for better formatting and functionality.
    """
    
    _configured = False
    
    @staticmethod
    def setup_logging(log_level: str = None, log_file: str = None) -> None:
        """
        Setup logging configuration.
        
        Args:
            log_level: Logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
            log_file: Path to log file
        """
        
        if LoggingConfig._configured:
            return
        
        log_level = log_level or settings.LOG_LEVEL
        log_file = log_file or settings.LOG_FILE
        
        # Remove default handler
        logger.remove()
        
        # Add console handler with color
        logger.add(
            sys.stderr,
            format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>",
            level=log_level,
            colorize=True
        )
        
        # Create logs directory if it doesn't exist
        log_path = Path(log_file)
        log_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Add file handler
        logger.add(
            log_file,
            format="{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {name}:{function}:{line} - {message}",
            level=log_level,
            rotation="500 MB",  # Rotate when file reaches 500MB
            retention="7 days"  # Keep logs for 7 days
        )
        
        LoggingConfig._configured = True
        logger.info(f"Logging configured - Level: {log_level}, File: {log_file}")
    
    @staticmethod
    def get_logger(name: str = None):
        """
        Get logger instance for a module.
        
        Args:
            name: Logger name (usually __name__)
        
        Returns:
            Configured logger instance
        """
        if not LoggingConfig._configured:
            LoggingConfig.setup_logging()
        
        return logger.bind(name=name or "app")

# Setup logging
LoggingConfig.setup_logging()
app_logger = LoggingConfig.get_logger(__name__)
app_logger.info("✓ Logging configuration completed")

## 3. Testing Configuration

In [ ]:
# Validate settings
app_logger.info("Validating configuration...")
errors = settings.validate()

if errors:
    print("\n❌ Configuration Errors:")
    for error in errors:
        print(f"  - {error}")
        app_logger.error(f"Config Error: {error}")
else:
    print("\n✅ Configuration validation passed!")
    app_logger.info("All configuration settings valid")

# Print settings summary
print("\n📋 Settings Summary:")
settings_dict = settings.to_dict()
for key, value in sorted(settings_dict.items()):
    if 'API_KEY' in key or 'TOKEN' in key:
        value = "***MASKED***" if value else "(not set)"
    print(f"  {key}: {value}")

## 4. Export Configuration (for use in other notebooks)

In [ ]:
# Save configuration for reuse
import pickle

config_data = {
    'settings': settings,
    'logger': app_logger,
    'LoggingConfig': LoggingConfig
}

with open('/tmp/config_module.pkl', 'wb') as f:
    pickle.dump(config_data, f)

print("✓ Configuration module exported to /tmp/config_module.pkl")
print("\nTo use in other notebooks:")
print("""import pickle
with open('/tmp/config_module.pkl', 'rb') as f:
    config_data = pickle.load(f)
    settings = config_data['settings']
    app_logger = config_data['logger']
    LoggingConfig = config_data['LoggingConfig']
""")

## 5. Summary

✅ **Configuration Module Completed**

### What was created:
1. **Settings Class** - Centralized configuration management
   - Loads from environment variables
   - Provides sensible defaults
   - Validation support

2. **LoggingConfig Class** - Professional logging setup
   - Console and file logging
   - Log rotation support
   - Colored output

### Next Steps:
- Run the utilities notebook to create helper functions
- Create the MAYINI LLM architecture
- Build the job scraper module
